### 07. AI-agent

- En AI-agent är ett system/program som kan agera autonomt för att uppnå ett mål
    - ta in information (input),
    - fatta beslut baserat om vad som ska göras (LLM står för logiken)
    - utföra åtgärder (använda 'tools').

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

![07 diagram](docs/07.drawio.png)

In [1]:
import os
import json

from dotenv import load_dotenv
import httpx

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
# obs: i detta exempel använder vi LangChain och LangGraph som är populära ramverk för bygga agenter (speciellt i python)
# i .net-världen är Semantic Kernel ett populärt alternativ

In [2]:
@tool
def get_location_coordinates(location: str) -> dict:
    """returns coordinates in latitude and longitude for the given location

    Args:
        location (str): the location

    Returns:
        dict: latitude and longitude
    """

    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"

    response = httpx.get(url)
    result = json.loads(response.text)
    location = result["results"][0]["geometry"]["location"]

    return {"latitude": location["lat"], "longitude": location["lng"]}

In [3]:
@tool
def get_current_temperature(latitude: str, longitude: str) -> dict:
    """return the current temperatire for the given coordinates

    Args:
        latitude (str): latitude
        longitude (str): longitude

    Returns:
        dict: the temperature
    """

    url = f"https://weather.googleapis.com/v1/currentConditions:lookup?key={API_KEY}&location.latitude={latitude}&location.longitude={longitude}"
    response = httpx.get(url)
    result = json.loads(response.text)

    return {
        "temperature": result["temperature"]["degrees"],
        "unit": result["temperature"]["unit"],
    }

In [4]:
memory = MemorySaver()
client = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=API_KEY)
tools = [get_location_coordinates, get_current_temperature]

agent_executor = create_react_agent(client, tools, checkpointer=memory)

In [5]:
config = {"configurable": {"thread_id": "abc123"}}

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what is the current temperature in moheda?")]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

what is the current temperature in moheda?
================================== Ai Message ==================================
Tool Calls:
  get_location_coordinates (b618e977-f359-41c7-8505-f2bd97a1d03b)
 Call ID: b618e977-f359-41c7-8505-f2bd97a1d03b
  Args:
    location: moheda
================================= Tool Message =================================
Name: get_location_coordinates

{"latitude": 57.0027566, "longitude": 14.5732973}
================================== Ai Message ==================================
Tool Calls:
  get_current_temperature (ec8ac3ff-34a1-49c2-a858-990a4e201de2)
 Call ID: ec8ac3ff-34a1-49c2-a858-990a4e201de2
  Args:
    latitude: 57.0027566
    longitude: 14.5732973
================================= Tool Message =================================
Name: get_current_temperature

{"temperature": 18, "unit": "CELSIUS"}
================================== Ai Message ================

In [6]:
config = {"configurable": {"thread_id": "abc1234"}}

for step in agent_executor.stream(
    {
        "messages": [
            HumanMessage(
                content="give me the current temperature in moheda and london?"
            )
        ]
    },
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

give me the current temperature in moheda and london?
================================== Ai Message ==================================
Tool Calls:
  get_location_coordinates (9e36911e-69e6-4d88-ad30-3e52137c2ff2)
 Call ID: 9e36911e-69e6-4d88-ad30-3e52137c2ff2
  Args:
    location: moheda
  get_location_coordinates (9dbc3e4a-2507-4303-b1c2-75fed0ac3818)
 Call ID: 9dbc3e4a-2507-4303-b1c2-75fed0ac3818
  Args:
    location: london
================================= Tool Message =================================
Name: get_location_coordinates

{"latitude": 51.5072178, "longitude": -0.1275862}
================================== Ai Message ==================================
Tool Calls:
  get_current_temperature (809d4110-73c1-41aa-9a0d-fbfdbe092e36)
 Call ID: 809d4110-73c1-41aa-9a0d-fbfdbe092e36
  Args:
    latitude: 57.0027566
    longitude: 14.5732973
  get_current_temperature (91c0f16e-7452-4d1b-a671-2591f5f8d